# Marathon Build Loops

## Integration Tests

In [ ]:
import logging
import os
from commons import JenkinsJob
from IPython.display import HTML, display


user, password = os.environ['AUTH'].split(":")

jobs = ["marathon-sandbox/job/marathon-loop-1.5", "marathon-sandbox/job/marathon-loop-1.6",
        "marathon-sandbox/job/marathon-loop-1.7", "marathon-sandbox/job/marathon-loop-master", ]

for j in jobs:
    job = JenkinsJob("jenkins.mesosphere.com/service/jenkins/view/Marathon/job", j, user, password)

    fails = await job.unique_fails()
    errors = await job.unique_errors()
    
    display(HTML('<h1>{}</h1>'.format(j)))
    print(fails.to_frame().to_html())
    print(errors.to_frame().to_html())

## System Integration Tests

In [ ]:
import os
from commons import JenkinsJob
from IPython.display import HTML, display


user, password = os.environ['AUTH'].split(":")

jobs = ["system-integration-tests/job/marathon-si-dcos-open/job/master",
        "system-integration-tests/job/marathon-si-dcos-permissive/job/master",
        "system-integration-tests/job/marathon-si-dcos-strict/job/master"]

for j in jobs:
    job = JenkinsJob("jenkins.mesosphere.com/service/jenkins/view/Marathon/job", j, user, password)

    fails = await job.unique_fails()
    errors = await job.unique_errors()
    
    display(HTML('<h1>{}</h1>'.format(j)))
    print(fails.to_frame().to_html())
    print(errors.to_frame().to_html())

In [1]:
import os
from commons import JenkinsJob
from IPython.display import HTML, display


user, password = os.environ['AUTH'].split(":")

job_name = "system-integration-tests/job/marathon-si-dcos-open/job/karsten%252Fdcos-terraform"

job = JenkinsJob("jenkins.mesosphere.com/service/jenkins/view/Marathon/job", job_name, user, password)

fails = await job.unique_fails()
errors = await job.unique_errors()
    
display(HTML('<h1>{}</h1>'.format(job)))
display(HTML(fails.to_frame().to_html()))
display(HTML(errors.to_frame().to_html()))

Could not fetch build 343
Traceback (most recent call last):
  File "/Users/kjeschkies/Projects/build-stats/commons.py", line 39, in grab_test
    resp.raise_for_status()
  File "/Users/kjeschkies/.local/share/virtualenvs/build-stats-iIh759Nx/lib/python3.6/site-packages/aiohttp/client_reqrep.py", line 942, in raise_for_status
    headers=self.headers)
aiohttp.client_exceptions.ClientResponseError: 404, message='Not Found'


,0
error,
AssertionError: Expected: eventually len: 0 but: after 60 retries len: was 1,1
AssertionError: Expected: eventually object with value id: not: 'sleep-app-02dd64eec8394d4098c848b7117ab272.instance-acba10cc-3d25-11e9-b989-a631adfbaf6a.$anon.1' but: after 30 retries value 'id' matched: 'sleep-app-02dd64eec8394d4098c848b7117ab272.instance-acba10cc-3d25-11e9-b989-a631adfbaf6a.$anon.1',1
AssertionError: Got the same task ID,1
"shakedown.errors.DCOSException: Server returned incorrect response type, expected application/vnd.dcos.package.list-response+json;charset=utf-8;version=v1 but got application/vnd.dcos.package.error+json;charset=utf-8;version=v1",2
test setup failed,1
test setup failure,4


In [29]:
df = await job.test_dataframe()
fail_statuses = ['FAILED', 'REGRESSION']
failed = df.loc[df.status.isin(fail_statuses)].set_index(['testsuite', 'testcase'], append=True)
display(failed)

status  \
build_id testsuite              testcase                                                    
380      test_marathon_root     test_event_channel                             REGRESSION   
377      test_marathon_root     test_event_channel                             REGRESSION   
         test_marathon_universe test_install_marathon                              FAILED   
376      test_marathon_universe test_install_marathon                          REGRESSION   
369      test_marathon_root     test_event_channel                             REGRESSION   
366      test_marathon_root     test_task_failure_recovers                     REGRESSION   
356      terraform              apply                                              FAILED   
352      test_marathon_root     test_restart_container_with_persistent_volume  REGRESSION   
338      test_marathon_root     test_pod_restarts_on_nonzero_exit_code         REGRESSION   
317      test_marathon_root     test_event_channel                             REGRESSION   

                                                                                                                           error  
build_id testsuite              testcase                                                                                          
380      test_marathon_root     test_event_channel                                                            test setup failure  
377      test_marathon_root     test_event_channel                                                            test setup failure  
         test_marathon_universe test_install_marathon                          shakedown.errors.DCOSException: Server returne...  
376      test_marathon_universe test_install_marathon                          shakedown.errors.DCOSException: Server returne...  
369      test_marathon_root     test_event_channel                                                            test setup failure  
366      test_marathon_root     test_task_failure_recovers                     AssertionError:  Expected: eventually object w...  
356      terraform              apply                                                                          test setup failed  
352      test_marathon_root     test_restart_container_with_persistent_volume  AssertionError:  Expected: eventually len: 0 b...  
338      test_marathon_root     test_pod_restarts_on_nonzero_exit_code                      AssertionError: Got the same task ID  
317      test_marathon_root     test_event_channel                                                            test setup failure

In [43]:
failed.groupby(level=['testsuite', 'testcase']).count()

status  \
testsuite              testcase                                                
terraform              apply                                               1   
test_marathon_root     test_event_channel                                  4   
                       test_pod_restarts_on_nonzero_exit_code              1   
                       test_restart_container_with_persistent_volume       1   
                       test_task_failure_recovers                          1   
test_marathon_universe test_install_marathon                               2   

                                                                      error  
testsuite              testcase                                              
terraform              apply                                              1  
test_marathon_root     test_event_channel                                 4  
                       test_pod_restarts_on_nonzero_exit_code             1  
                       test_restart_container_with_persistent_volume      1  
                       test_task_failure_recovers                         1  
test_marathon_universe test_install_marathon                              2

In [44]:
failed.groupby('error').count()

,status
error,
AssertionError: Expected: eventually len: 0 but: after 60 retries len: was 1,1
AssertionError: Expected: eventually object with value id: not: 'sleep-app-02dd64eec8394d4098c848b7117ab272.instance-acba10cc-3d25-11e9-b989-a631adfbaf6a.$anon.1' but: after 30 retries value 'id' matched: 'sleep-app-02dd64eec8394d4098c848b7117ab272.instance-acba10cc-3d25-11e9-b989-a631adfbaf6a.$anon.1',1
AssertionError: Got the same task ID,1
"shakedown.errors.DCOSException: Server returned incorrect response type, expected application/vnd.dcos.package.list-response+json;charset=utf-8;version=v1 but got application/vnd.dcos.package.error+json;charset=utf-8;version=v1",2
test setup failed,1
test setup failure,4
